### Carrega dados da camada Silver e Dimensões

In [0]:
from pyspark.sql import functions as f

In [0]:
# Silver
matricula_disciplina = spark.read.format("delta").load("/mnt/delta/silver/academico/matricula_disciplina")
matricula = spark.read.format("delta").load("/mnt/delta/silver/academico/matricula")

# Dimensões

dim_aluno = spark.read.format("delta").load("/mnt/delta/gold/academico/dimensao/dim_aluno")
dim_curso = spark.read.format("delta").load("/mnt/delta/gold/academico/dimensao/dim_curso")
dim_disciplina = spark.read.format("delta").load("/mnt/delta/gold/academico/dimensao/dim_disciplina")
dim_turma = spark.read.format("delta").load("/mnt/delta/gold/academico/dimensao/dim_turma")
dim_tempo = spark.table("default.dim_tempo")

In [0]:
df = (
    matricula.alias("matricula")
 
    .join(matricula_disciplina.alias("matricula_disciplina"), 
        ["id_matricula"]
        )
    
    .join(dim_tempo, 
        f.col("matricula.data_matricula") == f.col("dim_tempo.date")
        )
    
    .select(
        "id_matricula"
        , "id_aluno"
        , "id_turma"
        , "id_disciplina"
        , f.date_format("dim_tempo.date", "yyyyMMdd").alias("id_data_matricula")
        , "nota_final"
    )
)

In [0]:
df.write.format("delta").mode("overwrite").save(f"/mnt/delta/gold/academico/fato/fato_notas")